## Genetic Algorithms for Time Table Scheduling

In [24]:
import numpy as np
import pandas as pd
import random
import collections
import copy

# Load the data
courses_df = pd.read_csv('../Dataset/courses.csv')
student_courses_df = pd.read_csv('../Dataset/studentCourse.csv')
student_names_df = pd.read_csv('../Dataset/studentNames.csv')
teachers_df = pd.read_csv('../Dataset/teachers.csv')

# Define the hard constraints and soft constraints
hard_constraints = [
    "An exam will be scheduled for each course.",
    "A student cannot give more than one exam at a time.",
    "Exams will not be held on weekends.",
    "Exams must be between 9 am and 5 pm.",
    "Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.",
    "A teacher cannot invigilate two exams in a row."
]

soft_constraints = [
    "All students and teachers shall have a break on Friday from 1-2.",
    "A student shall not give more than one exam consecutively.",
    "If a student is enrolled in both MG and CS courses, it's preferred that MG course exam be held before the CS course exam.",
    "Faculty needs a two-hour break in the week."
]

# Define Gene Structure
class Exam:
    def __init__(self, course_code, start_time, classroom, invigilating_teacher):
        self.course_code = course_code
        self.start_time = start_time
        self.classroom = classroom
        self.invigilating_teacher = invigilating_teacher

    def __str__(self):
        return f"Exam(course_code={self.course_code}, start_time={self.start_time}, classroom={self.classroom}, invigilating_teacher={self.invigilating_teacher})"

# Define Chromosome Structure
class Schedule:
    def __init__(self, exams):
        self.exams = exams
        self.schfitness = 0 # fitness of Schedule
    
    def fitness(self):
        # Compute fitness based on hard and soft constraints
        hard_violations = 0
        soft_violations = 0

        print()
        print()
        l = []
        # c = []
        for x in self.exams:
            l.append(x.course_code)
        # for x in set(courses_df["Course Code"]):
        #     c.append(x)
        # print( sorted(l) )
        # print( sorted(c) )
        print(l)
        print()

        # print("\nhards", end=": ")

        # ! Hard constraints:
        
        # * 1. Check if a student has multiple exams at the same time
        student_schedule = collections.defaultdict(list)
        for exam in self.exams:
            for student in student_courses_df[student_courses_df["Course Code"] == exam.course_code]["Student Name"]:
                student_schedule[student].append(exam.start_time)
        
        for times in student_schedule.values():
            if len(times) != len(set(times)):
                hard_violations += 1


        # * 2. Check if teachers have multiple exams at the same time and each exam has an Invigilator
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for times in teacher_schedule.values():
            if len(times) != len(set(times)):
                hard_violations += 1
                # break # one penalty per wrong scheduling

        for exam in self.exams:
            if exam.invigilating_teacher == None or exam.invigilating_teacher == "":
                hard_violations += 1


        # * 3. An exam will be scheduled for each course
        exam_courses = []
        for exam in self.exams:
            exam_courses.append(exam.course_code)

        course_codes = list(set(courses_df["Course Code"].tolist()))
        for course in course_codes:
            if course not in exam_courses:
                hard_violations += 1
                # break # one penalty per wrong scheduling


        # * 4. Each exam must be held between 9 am and 5 pm.
        for exam in self.exams:
            # if exam.start_time.hour < 9 or exam.start_time.hour > 17:
            if exam.start_time.hour < 9 or exam.start_time.hour >= 17: 
                hard_violations += 1

            
        # * 5. A teacher cannot invigilate two exams in a row
        minimum_time_between_exams = 1
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for teacher, schedule in teacher_schedule.items():
            schedule.sort()
            for i in range(len(schedule) - 1):
                time_diff = (schedule[i + 1] - schedule[i]).total_seconds() // 3600  # Time difference in hours
                if time_diff < minimum_time_between_exams:  # Assuming a minimum time between exams
                    hard_violations += 1


        # * 6. Check if exams are not held on weekends
        for exam in self.exams:
            # if exam.start_time.weekday() in [5, 6]:
            if exam.start_time.weekday() in [6, 7]:
                hard_violations += 1


        # ! Soft constraints:
        
        # * 1. Ensure teachers have a break on Friday from 1-2
        friday_break_start = pd.to_datetime("13:00").time()
        friday_break_end = pd.to_datetime("14:00").time()
        
        for exam in self.exams:
            if exam.start_time.day == 5:  # Friday
                if exam.start_time.time() >= friday_break_start and exam.start_time.time() <= friday_break_end:
                    soft_violations += 1
        

        # * 2. Ensure students don't have consecutive exams
        for times in student_schedule.values():
            times = sorted(times)
            for i in range(1, len(times)):
                time_difference_seconds = (times[i] - times[i-1]).total_seconds()
                # if time_difference_seconds < 3600: # penalty for consequetive and same time
                if time_difference_seconds <= 3600 and time_difference_seconds > 0: # penalty for consequetive only
                    soft_violations += 1


        # * 3. If a student is enrolled in a MG course and a CS course, it is preferred that their
        for student, courses in student_courses_df.groupby("Student Name"):
            MG_flag = False
            CS_flag = False
            mg_code = None
            cs_code = None

            for course in courses["Course Code"].values:
                if "MG" in course:
                    MG_flag = True
                    mg_code = course
                elif "CS" in course:
                    CS_flag = True
                    cs_code = course
                
                if CS_flag and MG_flag: # required information found
                    break

            # Check if the student is enrolled in both MG and CS courses
            if CS_flag and MG_flag:

                mg_exam = None
                cs_exam = None
                for exam in self.exams:
                    if exam.course_code == mg_code:
                        mg_exam = exam
                    elif exam.course_code == cs_code:
                        cs_exam = exam

                # Ensure both exams exist
                if mg_exam is not None and cs_exam is not None:
                    # Check if CS exam is scheduled before MG exam
                    if cs_exam.start_time <= mg_exam.start_time:
                        # Increment soft violations if CS exam is scheduled before MG exam
                        soft_violations += 1  


        # * 4. 2 hours of break with half split of faculty
        break_start1 = pd.to_datetime("2024-04-01 12:00:00")  # Assuming break starts at noon on Monday
        break_end1 = break_start1 + pd.Timedelta(hours=1)  # Assuming one hour break
        break_start2 = pd.to_datetime("2024-04-04 12:00:00")  # Another break slot on Thursday
        break_end2 = break_start2 + pd.Timedelta(hours=1)  # Assuming one hour break
        available_teachers = teachers_df["Names"].tolist()

        # Count the number of teachers available in each break slot
        teachers_available_slot1 = 0
        teachers_available_slot2 = 0

        for teacher in available_teachers:
            available_slot1 = True
            available_slot2 = True
            for exam_time in teacher_schedule[teacher]:
                
                if exam_time >= break_start1 and exam_time <= break_end1: # check if teacher is free in both times slots
                    available_slot1 = False
                if exam_time >= break_start2 and exam_time <= break_end2:
                    available_slot2 = False

            if teachers_available_slot2 >= teachers_available_slot1: # if one time slot has more teachers than the other, try filling the other timeslot first to maintain balance.
                if available_slot1:
                    teachers_available_slot1 += 1
                elif available_slot2:
                    teachers_available_slot2 += 1
            else:
                if available_slot2:
                    teachers_available_slot2 += 1
                elif available_slot1:
                    teachers_available_slot1 += 1


        # Ensure at least half the faculty is free in one slot and the rest in the other slot
        if not ( (len(available_teachers) // 2) + 2 > teachers_available_slot1 >= (len(available_teachers) // 2) or (len(available_teachers) // 2) + 2 > teachers_available_slot2 >= (len(available_teachers) // 2) ) and \
            ((teachers_available_slot1) + (teachers_available_slot2) ) != len(available_teachers):
            soft_violations += 1

        
        # Compute the total fitness
        total_violations = hard_violations + 0.5 * soft_violations  # Soft violations have less weight

        return -total_violations
         
    def constraints_dissatisfied(self):
        # store dissatisfied constraints
        hard_constraints_dissatisfied = set()
        soft_constraints_dissatisfied = set()


        # ! Hard constraints:

        # * 1. Check if a student has multiple exams at the same time
        student_schedule = collections.defaultdict(list)
        for exam in self.exams:
            for student in student_courses_df[student_courses_df["Course Code"] == exam.course_code]["Student Name"]:
                student_schedule[student].append(exam.start_time)
        
        for times in student_schedule.values():
            if len(times) != len(set(times)):
                hard_constraints_dissatisfied.add(hard_constraints[1])
                break


        # * 2. Check if teachers have multiple exams at the same time and each exam has an invigilator
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for times in teacher_schedule.values():
            if len(times) != len(set(times)):
                hard_constraints_dissatisfied.add(hard_constraints[4])
                break

        for exam in self.exams:
            if exam.invigilating_teacher == None or exam.invigilating_teacher == "":
                hard_constraints_dissatisfied.add(hard_constraints[4])
                break
                


        # * 3. An exam will be scheduled for each course
        exam_courses = []
        for exam in self.exams:
            exam_courses.append(exam.course_code)

        course_codes = list(set(courses_df["Course Code"].tolist()))
        for course in course_codes:
            if course not in exam_courses:
                hard_constraints_dissatisfied.add(hard_constraints[0])
                break


        # * 4. Each exam must be held between 9 am and 5 pm.
        for exam in self.exams:
            # if exam.start_time.hour < 9 or exam.start_time.hour > 17:
            if exam.start_time.hour < 9 or exam.start_time.hour >= 17: 
                hard_constraints_dissatisfied.add(hard_constraints[3])
                break

            
        # * 5. A teacher cannot invigilate two exams in a row
        minimum_time_between_exams = 1
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for teacher, schedule in teacher_schedule.items():
            schedule.sort()
            for i in range(len(schedule) - 1):
                time_diff = (schedule[i + 1] - schedule[i]).total_seconds() // 3600  # Time difference in hours
                if time_diff < minimum_time_between_exams:  # Assuming a minimum time between exams
                    hard_constraints_dissatisfied.add(hard_constraints[5])
                    break


        # * 6. Check if exams are not held on weekends
        for exam in self.exams:
            # if exam.start_time.weekday() in [5, 6]:
            if exam.start_time.weekday() in [6, 7]:
                hard_constraints_dissatisfied.add(hard_constraints[2])
                break


        # ! Soft constraints:
        
        # * 1. Ensure teachers have a break on Friday from 1-2
        friday_break_start = pd.to_datetime("13:00").time()
        friday_break_end = pd.to_datetime("14:00").time()
        
        for exam in self.exams:
            if exam.start_time.day == 5:  # Friday
                if exam.start_time.time() >= friday_break_start and exam.start_time.time() <= friday_break_end:
                    soft_constraints_dissatisfied.add(soft_constraints[0])
                    break
        
        # * 2. Ensure students don't have consecutive exams
        for times in student_schedule.values():
            times = sorted(times)
            for i in range(1, len(times)):
                time_difference_seconds = (times[i] - times[i-1]).total_seconds()
                # if time_difference_seconds < 3600: # penalty for consequetive and same time
                if time_difference_seconds <= 3600 and time_difference_seconds > 0: # penalty for consequetive only
                    soft_constraints_dissatisfied.add(soft_constraints[1])
                    break


        # * 3. If a student is enrolled in a MG course and a CS course, it is preferred that their
        for student, courses in student_courses_df.groupby("Student Name"):
            MG_flag = False
            CS_flag = False
            mg_code = None
            cs_code = None

            for course in courses["Course Code"].values:
                if "MG" in course:
                    MG_flag = True
                    mg_code = course
                elif "CS" in course:
                    CS_flag = True
                    cs_code = course
                
                if CS_flag and MG_flag: # required information found
                    break

            # Check if the student is enrolled in both MG and CS courses
            if CS_flag and MG_flag:

                mg_exam = None
                cs_exam = None
                for exam in self.exams:
                    if exam.course_code == mg_code:
                        mg_exam = exam
                    elif exam.course_code == cs_code:
                        cs_exam = exam

                # Ensure both exams exist
                if mg_exam is not None and cs_exam is not None:
                    # Check if CS exam is scheduled before MG exam
                    if cs_exam.start_time <= mg_exam.start_time:
                        # Increment soft violations if CS exam is scheduled before MG exam
                        soft_constraints_dissatisfied.add(soft_constraints[2])
                        break


        # * 4. 2 hours of break with half split of faculty
        break_start1 = pd.to_datetime("2024-04-01 12:00:00")  # Assuming break starts at noon on Monday
        break_end1 = break_start1 + pd.Timedelta(hours=1)  # Assuming one hour break
        break_start2 = pd.to_datetime("2024-04-04 12:00:00")  # Another break slot on Thursday
        break_end2 = break_start2 + pd.Timedelta(hours=1)  # Assuming one hour break
        available_teachers = teachers_df["Names"].tolist()

        # Count the number of teachers available in each break slot
        teachers_available_slot1 = 0
        teachers_available_slot2 = 0

        for teacher in available_teachers:
            available_slot1 = True
            available_slot2 = True
            for exam_time in teacher_schedule[teacher]:
                
                if exam_time >= break_start1 and exam_time <= break_end1: # check if teacher is free in both times slots
                    available_slot1 = False
                if exam_time >= break_start2 and exam_time <= break_end2:
                    available_slot2 = False

            if teachers_available_slot2 >= teachers_available_slot1: # if one time slot has more teachers than the other, try filling the other timeslot first to maintain balance.
                if available_slot1:
                    teachers_available_slot1 += 1
                elif available_slot2:
                    teachers_available_slot2 += 1
            else:
                if available_slot2:
                    teachers_available_slot2 += 1
                elif available_slot1:
                    teachers_available_slot1 += 1


        # Ensure at least half the faculty is free in one slot and the rest in the other slot
        if not ( (len(available_teachers) // 2) + 2 > teachers_available_slot1 >= (len(available_teachers) // 2) or (len(available_teachers) // 2) + 2 > teachers_available_slot2 >= (len(available_teachers) // 2) ) and \
            ((teachers_available_slot1) + (teachers_available_slot2) ) != len(available_teachers):
            soft_constraints_dissatisfied.add(soft_constraints[3])
        
        constraints = [ hard_constraints_dissatisfied, soft_constraints_dissatisfied ] # combine both constraints into a single list

        return constraints
 
    def __str__(self):
        return '\n'.join(str(exam) for exam in self.exams) # used to display the Object

# Step 4: Define Genetic Operators
def crossover(parent1, parent2):
    # One-point crossover
    cross_point = random.randint(1, len(parent1.exams) - 1)
    child1 = parent1.exams[:cross_point] + parent2.exams[cross_point:]
    child2 = parent2.exams[:cross_point] + parent1.exams[cross_point:]

    return Schedule(child1), Schedule(child2)

def crossover2(parent1, parent2):
    # multi-point crossover

    num_exams = len(parent1.exams)

    # Select two random crossover points
    point1 = random.randint(1, num_exams - 2)
    point2 = random.randint(point1 + 1, num_exams - 1)

    # Create children by swapping segments
    child1_exams = parent1.exams[:point1] + parent2.exams[point1:point2] + parent1.exams[point2:]
    child2_exams = parent2.exams[:point1] + parent1.exams[point1:point2] + parent2.exams[point2:]

    # Ensure no duplicates and all required exams are included
    # child1_exams = list(set(child1_exams))
    # child2_exams = list(set(child2_exams))

    return Schedule(child1_exams), Schedule(child2_exams)

def crossover3(parent1, parent2):
    num_exams = len(parent1.exams)

    # Select a random subset of exams from parent1
    subset_start = random.randint(0, num_exams // 2 - 1)
    subset_end = random.randint(num_exams // 2, num_exams - 1)

    # Create a subset of exams to retain
    child1_subset = parent1.exams[subset_start:subset_end]
    child2_subset = parent2.exams[subset_start:subset_end]

    # Fill in the remaining exams from the other parent, maintaining the order
    child1_remaining = [exam for exam in parent2.exams if exam not in child1_subset]
    child2_remaining = [exam for exam in parent1.exams if exam not in child2_subset]

    child1_exams = child1_remaining[:subset_start] + child1_subset + child1_remaining[subset_start:]
    child2_exams = child2_remaining[:subset_start] + child2_subset + child2_remaining[subset_start:]

    return Schedule(child1_exams), Schedule(child2_exams)

def crossover4(parent1, parent2):

    offspring1 = copy.deepcopy(parent1.exams)
    offspring2 = copy.deepcopy(parent2.exams)

    # Swap genes between the two parents at the chosen indices
    # for i in range(0, len(parent1.exams) - 1):
    for i in range(0, len(parent1.exams)):
        if random.random() <= 0.5:
            offspring1[i], offspring2[i] = offspring2[i], offspring1[i]

    return Schedule(offspring1), Schedule(offspring2)

def crossover5(parent1, parent2):
    
    offspring1 = copy.deepcopy(parent1.exams)
    offspring2 = copy.deepcopy(parent2.exams)

    for i in range(0, len(parent1.exams)):

        if random.random() <= 0.5:

            # Choose two random indices for gene swapping
            index1 = random.randint(0, len(parent1.exams) - 1)
            index2 = random.randint(0, len(parent1.exams) - 1)

            offspring1[index1], offspring2[index2] = offspring2[index1], offspring1[index2]
            # offspring1[index1], offspring2[index2] = offspring2[index2], offspring1[index1]
            # offspring1[index1], offspring2[index1] = offspring2[index1], offspring1[index1]

    return Schedule(offspring1), Schedule(offspring2)
    

def mutation(schedule):
    # Swap two exams as a simple mutation strategy
    index1 = random.randint(0, len(schedule.exams) - 1)
    index2 = random.randint(0, len(schedule.exams) - 1)
    
    schedule.exams[index1], schedule.exams[index2] = schedule.exams[index2], schedule.exams[index1]
    return schedule

def mutation2(schedule):
    index1 = random.randint(0, len(schedule.exams) - 1)
    index2 = random.randint(0, len(schedule.exams) - 1)

    # Remove exam from original position and insert into new position
    exam = schedule.exams.pop(index1)
    schedule.exams.insert(index2, exam)

    return schedule

def mutation3(schedule):

    mutated_chromosome = copy.deepcopy(schedule.exams)
    # Select a subset of genes
    subset_start = random.randint(0, len(mutated_chromosome) - 1)
    subset_end = random.randint(subset_start, len(mutated_chromosome))
    subset = mutated_chromosome[subset_start:subset_end]
    # Invert the subset
    inverted_subset = subset[::-1]
    # Update the chromosome with the inverted subset
    mutated_chromosome[subset_start:subset_end] = inverted_subset

    return Schedule(mutated_chromosome)

def mutation4(schedule):

    mutated_chromosome = copy.deepcopy(schedule.exams)
    # Select a subset of genes
    subset_size = random.randint(1, len(mutated_chromosome))
    subset_indices = random.sample(range(len(mutated_chromosome)), subset_size)
    subset_values = [mutated_chromosome[i] for i in subset_indices]
    # Shuffle or scramble the values of the subset randomly
    random.shuffle(subset_values)
    # Update the chromosome with the scrambled subset
    for i, index in enumerate(subset_indices):
        mutated_chromosome[index] = subset_values[i]

    return Schedule(mutated_chromosome)


def roulette_wheel_selection(population):
    fitnesses = np.array([sched.fitness() for sched in population])
    fitnesses = fitnesses - np.min(fitnesses) + 1  # Normalization to avoid division by zero later 
    
    total_fitness = np.sum(fitnesses)
    selection_probabilities = fitnesses / total_fitness # Calculate selection probabilities
    
    parents = random.choices(population, selection_probabilities, k=2) # Choose parents randomly based on selection probability
    return parents[0], parents[1]

def roulette_wheel_selection2(population):
    total_fitness = sum(x.fitness() for x in population)
    parents = []
    loop_counter = 0

    while loop_counter < 2:
        pick = random.uniform(0,total_fitness) # pick random threshold
        current =0
        for x in population: # loop through population

            fitness = x.fitness()
            if current + fitness >= pick and x not in parents: # if threshold reached, exit loop with parent
                parents.append(x)
                loop_counter += 1
                break
                
            current += fitness

    return parents[0], parents[1]


# Create initial population and simulate evolution
def create_initial_population(num_individuals, available_times, available_classrooms, available_teachers):
    population = []
    
    for _ in range(num_individuals):
        exams = []
        
        for _, course in courses_df.iterrows():
            flag = False
            course_code = course["Course Code"] # Get the course code

            for exam in exams: # If course code already exists, continue
                if course_code == exam.course_code:
                    flag = True

            if flag:
                continue

            # Set details for exam
            start_time = random.choice(available_times) 
            classroom = random.choice(available_classrooms)
            teacher = random.choice(available_teachers)
            
            # Store exams in a list
            exams.append(Exam(course_code, start_time, classroom, teacher))
        
        # Store exams list in a Schedule object
        population.append(Schedule(exams))
    
    return population

def run_genetic_algorithm(num_generations, population_size, mutation_rate, crossover_rate):
    
    # Define available time slots
    time_slots = [pd.to_datetime(f"2024-04-{i} {j}:00:00") for i in range(1, 6) for j in range(9, 17)] # Define available days
    available_classrooms = [f"C{str(x).zfill(3)}" for x in range(301, 311)] # Define available time slots
    available_teachers = list(teachers_df["Names"]) # Define available teachers
    
    # Create initial population
    population = create_initial_population(population_size, time_slots, available_classrooms, available_teachers)

    # Generations loop
    for generation in range(num_generations):

        for chromosome in population:
            chromosome.schfitness = chromosome.fitness() # Initialize fitness of population

        print("Population 1: ")
        for x in population:
            print(x.schfitness, end=', ')
        print()
        
        for _ in range( population_size // 2 ): # Loop for half of population

            #  Parent Selection
            parent1, parent2 = roulette_wheel_selection(population)
            
            # Crossover
            if random.random() < crossover_rate:
                child1, child2 = crossover4(parent1, parent2)
            else:
                child1, child2 = parent1, parent2

            # Mutation
            if random.random() < mutation_rate:
                child1 = mutation2(child1)
            if random.random() < mutation_rate:
                child2 = mutation2(child2)

            # Initalize children fitness
            child1.schfitness = child1.fitness()
            child2.schfitness = child2.fitness()
            
            # Append children
            population.append(child1)
            population.append(child2)

            population.sort(key=lambda x: x.schfitness, reverse=True) # Sort Schedules by fitness
            population = population[:population_size]   # Slice population to population_size
        
        print(f"Population {generation+1}:") 
        for x in population:
            print(x.schfitness, end=", ")
        print()
        # Print best solution fitness at each generation 

        best_fitness = max(sched.fitness() for sched in population) # Get best solution fitness in current population
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")     # Display best solution fitness
    
    # Return the best solution found
    best_schedule = max(population, key=lambda sched: sched.fitness())
    
    return best_schedule


num_generations = 30
population_size = 10
mutation_rate = 0.2
crossover_rate = 0.95

best_schedule = run_genetic_algorithm(num_generations, population_size, mutation_rate, crossover_rate)

print("Best schedule found:")
print(best_schedule)




['CS217', 'EE227', 'CS211', 'SE110', 'CS118', 'CS219', 'CS220', 'CS302', 'CY2012', 'CS307', 'CS328', 'EE229', 'AI2011', 'DS3011', 'CS218', 'MT224', 'SS113', 'MG220', 'MG223', 'SS111', 'SS152', 'SS118', 'MT205']



['CS217', 'EE227', 'CS211', 'SE110', 'CS118', 'CS219', 'CS220', 'CS302', 'CY2012', 'CS307', 'CS328', 'EE229', 'AI2011', 'DS3011', 'CS218', 'MT224', 'SS113', 'MG220', 'MG223', 'SS111', 'SS152', 'SS118', 'MT205']



['CS217', 'EE227', 'CS211', 'SE110', 'CS118', 'CS219', 'CS220', 'CS302', 'CY2012', 'CS307', 'CS328', 'EE229', 'AI2011', 'DS3011', 'CS218', 'MT224', 'SS113', 'MG220', 'MG223', 'SS111', 'SS152', 'SS118', 'MT205']



['CS217', 'EE227', 'CS211', 'SE110', 'CS118', 'CS219', 'CS220', 'CS302', 'CY2012', 'CS307', 'CS328', 'EE229', 'AI2011', 'DS3011', 'CS218', 'MT224', 'SS113', 'MG220', 'MG223', 'SS111', 'SS152', 'SS118', 'MT205']



['CS217', 'EE227', 'CS211', 'SE110', 'CS118', 'CS219', 'CS220', 'CS302', 'CY2012', 'CS307', 'CS328', 'EE229', 'AI2011', 'DS3011', 'CS218', 'MT

## Display the best schedule

In [9]:
import pandas as pd

# Initialize lists to store exam details
course_codes = []
start_times = []
classrooms = []
invigilating_teachers = []

# Extract exam details from the best_schedule
for exam in best_schedule.exams:
    course_codes.append(exam.course_code)
    start_times.append(exam.start_time)
    classrooms.append(exam.classroom)
    invigilating_teachers.append(exam.invigilating_teacher)

# Create a DataFrame to represent the schedule
schedule_df = pd.DataFrame({
    "Start Time": start_times,
    "Classroom": classrooms,
    "Invigilating Teacher": invigilating_teachers
}, index=course_codes)  # Set "Course Code" as index

# Set the name of the index column
schedule_df = schedule_df.rename_axis("Course Code")

# Display the DataFrame
schedule_df.head(100)


,Start Time,Classroom,Invigilating Teacher
Course Code,,,
CS217,2024-04-04 14:00:00,C301,Ayesha Bano
EE227,2024-04-03 15:00:00,C310,Umair Arshad
CS211,2024-04-05 12:00:00,C306,Naveed Ahmad
SE110,2024-04-05 10:00:00,C301,Kashif Munir
CS118,2024-04-04 13:00:00,C305,Usman Rashid
CS219,2024-04-01 16:00:00,C309,Shehreyar Rashid
CS220,2024-04-01 14:00:00,C310,Shams Farooq
CS302,2024-04-01 09:00:00,C303,Sadia Nauman
CS328,2024-04-05 09:00:00,C304,Bilal Khalid


## Display Constraints 

In [10]:
hard_const, soft_const = best_schedule.constraints_dissatisfied()

print("Hard Constraints Satisfied: ")
print('---------------------------------------')
for index, const in enumerate(hard_constraints):
    if const not in hard_const:
        print(f'{index+1}.', const)

print()

print("Soft Constraints Satisfied: ")
print('---------------------------------------')
for index, const in enumerate(soft_constraints):
    if const not in soft_const:
        print(f'{index+1}.', const)


Hard Constraints Satisfied: 
---------------------------------------
1. An exam will be scheduled for each course.
3. Exams will not be held on weekends.
4. Exams must be between 9 am and 5 pm.
5. Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
6. A teacher cannot invigilate two exams in a row.

Soft Constraints Satisfied: 
---------------------------------------
1. All students and teachers shall have a break on Friday from 1-2.
4. Faculty needs a two-hour break in the week.


In [11]:
hard_const, soft_const = best_schedule.constraints_dissatisfied()

print("Hard Constraints Dissatisfied: ")
print('---------------------------------------')
for index, const in enumerate(hard_const):
        print(f'{index+1}.', const)

print()

print("Soft Constraints Dissatisfied: ")
print('---------------------------------------')
for index, const in enumerate(soft_const):
        print(f'{index+1}.', const)


Hard Constraints Dissatisfied: 
---------------------------------------
1. A student cannot give more than one exam at a time.

Soft Constraints Dissatisfied: 
---------------------------------------
1. If a student is enrolled in both MG and CS courses, it's preferred that MG course exam be held before the CS course exam.
2. A student shall not give more than one exam consecutively.


## 7 Constraints

In [19]:
import numpy as np
import pandas as pd
import random
import itertools
import collections
import copy

# Load the data
courses_df = pd.read_csv('../Dataset/courses.csv')
student_courses_df = pd.read_csv('../Dataset/studentCourse.csv')
student_names_df = pd.read_csv('../Dataset/studentNames.csv')
teachers_df = pd.read_csv('../Dataset/teachers.csv')

# Define the hard constraints and soft constraints
hard_constraints = [
    "An exam will be scheduled for each course.",
    "A student cannot give more than one exam at a time.",
    "Exams will not be held on weekends.",
    "Exams must be between 9 am and 5 pm.",
    "Each exam must be invigilated by a teacher.",
    "A teacher cannot invigilate two exams at the same time.",
    "A teacher cannot invigilate two exams in a row."
]

soft_constraints = [
    "All students and teachers shall have a break on Friday from 1-2.",
    "A student shall not give more than one exam consecutively.",
    "If a student is enrolled in both MG and CS courses, it's preferred that MG course exam be held before the CS course exam.",
    "Faculty needs a two-hour break in the week."
]

# Define Gene Structure
class Exam:
    def __init__(self, course_code, start_time, classroom, invigilating_teacher):
        self.course_code = course_code
        self.start_time = start_time
        self.classroom = classroom
        self.invigilating_teacher = invigilating_teacher

    def __str__(self):
        return f"Exam(course_code={self.course_code}, start_time={self.start_time}, classroom={self.classroom}, invigilating_teacher={self.invigilating_teacher})"

# Define Chromosome Structure
class Schedule:
    def __init__(self, exams):
        self.exams = exams
        self.schfitness = 0 # fitness of Schedule
    
    def fitness(self):
        # Compute fitness based on hard and soft constraints
        hard_violations = 0
        soft_violations = 0

        # ! Hard constraints:
        
        # * 1. Check if a student has multiple exams at the same time
        student_schedule = collections.defaultdict(list)
        for exam in self.exams:
            for student in student_courses_df[student_courses_df["Course Code"] == exam.course_code]["Student Name"]:
                student_schedule[student].append(exam.start_time)
        
        for times in student_schedule.values():
            if len(times) != len(set(times)):
                hard_violations += 1


        # * 2. Check if teachers have multiple exams at the same time
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for times in teacher_schedule.values():
            if len(times) != len(set(times)):
                hard_violations += 1
                # break # one penalty per wrong scheduling

        # * 3. Each exam has an Invigilator 
        for exam in self.exams:
            if exam.invigilating_teacher == None or exam.invigilating_teacher == "":
                hard_violations += 1


        # * 4. An exam will be scheduled for each course
        exam_courses = []
        for exam in self.exams:
            exam_courses.append(exam.course_code)

        course_codes = list(set(courses_df["Course Code"].tolist()))
        for course in course_codes:
            if course not in exam_courses:
                hard_violations += 1
                # break # one penalty per wrong scheduling


        # * 5. Each exam must be held between 9 am and 5 pm.
        for exam in self.exams:
            # if exam.start_time.hour < 9 or exam.start_time.hour > 17:
            if exam.start_time.hour < 9 or exam.start_time.hour >= 17: 
                hard_violations += 1

            
        # * 6. A teacher cannot invigilate two exams in a row
        minimum_time_between_exams = 1
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for teacher, schedule in teacher_schedule.items():
            schedule.sort()
            for i in range(len(schedule) - 1):
                time_diff = (schedule[i + 1] - schedule[i]).total_seconds() // 3600  # Time difference in hours
                if time_diff < minimum_time_between_exams:  # Assuming a minimum time between exams
                    hard_violations += 1


        # * 7. Check if exams are not held on weekends
        for exam in self.exams:
            # if exam.start_time.weekday() in [5, 6]:
            if exam.start_time.weekday() in [6, 7]:
                hard_violations += 1


        # ! Soft constraints:
        
        # * 1. Ensure teachers have a break on Friday from 1-2
        friday_break_start = pd.to_datetime("13:00").time()
        friday_break_end = pd.to_datetime("14:00").time()
        
        for exam in self.exams:
            if exam.start_time.day == 5:  # Friday
                if exam.start_time.time() >= friday_break_start and exam.start_time.time() <= friday_break_end:
                    soft_violations += 1
        

        # * 2. Ensure students don't have consecutive exams
        for times in student_schedule.values():
            times = sorted(times)
            for i in range(1, len(times)):
                time_difference_seconds = (times[i] - times[i-1]).total_seconds()
                # if time_difference_seconds < 3600: # penalty for consequetive and same time
                if time_difference_seconds <= 3600 and time_difference_seconds > 0: # penalty for consequetive only
                    soft_violations += 1


        # * 3. If a student is enrolled in a MG course and a CS course, it is preferred that their
        for student, courses in student_courses_df.groupby("Student Name"):
            MG_flag = False
            CS_flag = False
            mg_code = None
            cs_code = None

            for course in courses["Course Code"].values:
                if "MG" in course:
                    MG_flag = True
                    mg_code = course
                elif "CS" in course:
                    CS_flag = True
                    cs_code = course
                
                if CS_flag and MG_flag: # required information found
                    break

            # Check if the student is enrolled in both MG and CS courses
            if CS_flag and MG_flag:

                mg_exam = None
                cs_exam = None
                for exam in self.exams:
                    if exam.course_code == mg_code:
                        mg_exam = exam
                    elif exam.course_code == cs_code:
                        cs_exam = exam

                # Ensure both exams exist
                if mg_exam is not None and cs_exam is not None:
                    # Check if CS exam is scheduled before MG exam
                    if cs_exam.start_time <= mg_exam.start_time:
                        # Increment soft violations if CS exam is scheduled before MG exam
                        soft_violations += 1  


        # * 4. 2 hours of break with half split of faculty
        break_start1 = pd.to_datetime("2024-04-01 12:00:00")  # Assuming break starts at noon on Monday
        break_end1 = break_start1 + pd.Timedelta(hours=1)  # Assuming one hour break
        break_start2 = pd.to_datetime("2024-04-04 12:00:00")  # Another break slot on Thursday
        break_end2 = break_start2 + pd.Timedelta(hours=1)  # Assuming one hour break
        available_teachers = teachers_df["Names"].tolist()

        # Count the number of teachers available in each break slot
        teachers_available_slot1 = 0
        teachers_available_slot2 = 0

        for teacher in available_teachers:
            available_slot1 = True
            available_slot2 = True
            for exam_time in teacher_schedule[teacher]:
                
                if exam_time >= break_start1 and exam_time <= break_end1: # check if teacher is free in both times slots
                    available_slot1 = False
                if exam_time >= break_start2 and exam_time <= break_end2:
                    available_slot2 = False

            if teachers_available_slot2 >= teachers_available_slot1: # if one time slot has more teachers than the other, try filling the other timeslot first to maintain balance.
                if available_slot1:
                    teachers_available_slot1 += 1
                elif available_slot2:
                    teachers_available_slot2 += 1
            else:
                if available_slot2:
                    teachers_available_slot2 += 1
                elif available_slot1:
                    teachers_available_slot1 += 1


        # Ensure at least half the faculty is free in one slot and the rest in the other slot
        if not ( (len(available_teachers) // 2) + 2 > teachers_available_slot1 >= (len(available_teachers) // 2) or (len(available_teachers) // 2) + 2 > teachers_available_slot2 >= (len(available_teachers) // 2) ) and \
            ((teachers_available_slot1) + (teachers_available_slot2) ) != len(available_teachers):
            soft_violations += 1

        
        # Compute the total fitness
        total_violations = hard_violations + 0.5 * soft_violations  # Soft violations have less weight

        return -total_violations
         
    def constraints_dissatisfied(self):
        # store dissatisfied constraints
        hard_constraints_dissatisfied = set()
        soft_constraints_dissatisfied = set()


        # ! Hard constraints:

        # * 1. Check if a student has multiple exams at the same time
        student_schedule = collections.defaultdict(list)
        for exam in self.exams:
            for student in student_courses_df[student_courses_df["Course Code"] == exam.course_code]["Student Name"]:
                student_schedule[student].append(exam.start_time)
        
        for times in student_schedule.values():
            if len(times) != len(set(times)):
                hard_constraints_dissatisfied.add(hard_constraints[1])
                break


        # * 2. Check if teachers have multiple exams at the same time
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for times in teacher_schedule.values():
            if len(times) != len(set(times)):
                hard_constraints_dissatisfied.add(hard_constraints[5])
                break

        # * 3. Each exam has an invigilator         
        for exam in self.exams:
            if exam.invigilating_teacher == None or exam.invigilating_teacher == "":
                hard_constraints_dissatisfied.add(hard_constraints[4])
                break
                


        # * 4. An exam will be scheduled for each course
        exam_courses = []
        for exam in self.exams:
            exam_courses.append(exam.course_code)

        course_codes = list(set(courses_df["Course Code"].tolist()))
        for course in course_codes:
            if course not in exam_courses:
                hard_constraints_dissatisfied.add(hard_constraints[0])
                break


        # * 5. Each exam must be held between 9 am and 5 pm.
        for exam in self.exams:
            # if exam.start_time.hour < 9 or exam.start_time.hour > 17:
            if exam.start_time.hour < 9 or exam.start_time.hour >= 17: 
                hard_constraints_dissatisfied.add(hard_constraints[3])
                break

            
        # * 6. A teacher cannot invigilate two exams in a row
        minimum_time_between_exams = 1
        teacher_schedule = collections.defaultdict(list)
        for exam in self.exams:
            teacher_schedule[exam.invigilating_teacher].append(exam.start_time)

        for teacher, schedule in teacher_schedule.items():
            schedule.sort()
            for i in range(len(schedule) - 1):
                time_diff = (schedule[i + 1] - schedule[i]).total_seconds() // 3600  # Time difference in hours
                if time_diff < minimum_time_between_exams:  # Assuming a minimum time between exams
                    hard_constraints_dissatisfied.add(hard_constraints[6])
                    break


        # * 7. Check if exams are not held on weekends
        for exam in self.exams:
            if exam.start_time.weekday() in [5, 6]:
                hard_constraints_dissatisfied.add(hard_constraints[2])
                break


        # ! Soft constraints:
        
        # * 1. Ensure teachers have a break on Friday from 1-2
        friday_break_start = pd.to_datetime("13:00").time()
        friday_break_end = pd.to_datetime("14:00").time()
        
        for exam in self.exams:
            if exam.start_time.day == 5:  # Friday
                if exam.start_time.time() >= friday_break_start and exam.start_time.time() <= friday_break_end:
                    soft_constraints_dissatisfied.add(soft_constraints[0])
                    break
        
        # * 2. Ensure students don't have consecutive exams
        for times in student_schedule.values():
            times = sorted(times)
            for i in range(1, len(times)):
                time_difference_seconds = (times[i] - times[i-1]).total_seconds()
                # if time_difference_seconds < 3600: # penalty for consequetive and same time
                if time_difference_seconds <= 3600 and time_difference_seconds > 0: # penalty for consequetive only
                    soft_constraints_dissatisfied.add(soft_constraints[1])
                    break


        # * 3. If a student is enrolled in a MG course and a CS course, it is preferred that their
        for student, courses in student_courses_df.groupby("Student Name"):
            MG_flag = False
            CS_flag = False
            mg_code = None
            cs_code = None

            for course in courses["Course Code"].values:
                if "MG" in course:
                    MG_flag = True
                    mg_code = course
                elif "CS" in course:
                    CS_flag = True
                    cs_code = course
                
                if CS_flag and MG_flag: # required information found
                    break

            # Check if the student is enrolled in both MG and CS courses
            if CS_flag and MG_flag:

                mg_exam = None
                cs_exam = None
                for exam in self.exams:
                    if exam.course_code == mg_code:
                        mg_exam = exam
                    elif exam.course_code == cs_code:
                        cs_exam = exam

                # Ensure both exams exist
                if mg_exam is not None and cs_exam is not None:
                    # Check if CS exam is scheduled before MG exam
                    if cs_exam.start_time <= mg_exam.start_time:
                        # Increment soft violations if CS exam is scheduled before MG exam
                        soft_constraints_dissatisfied.add(soft_constraints[2])
                        break


        # * 4. 2 hours of break with half split of faculty
        break_start1 = pd.to_datetime("2024-04-01 12:00:00")  # Assuming break starts at noon on Monday
        break_end1 = break_start1 + pd.Timedelta(hours=1)  # Assuming one hour break
        break_start2 = pd.to_datetime("2024-04-04 12:00:00")  # Another break slot on Thursday
        break_end2 = break_start2 + pd.Timedelta(hours=1)  # Assuming one hour break
        available_teachers = teachers_df["Names"].tolist()

        # Count the number of teachers available in each break slot
        teachers_available_slot1 = 0
        teachers_available_slot2 = 0

        for teacher in available_teachers:
            available_slot1 = True
            available_slot2 = True
            for exam_time in teacher_schedule[teacher]:
                
                if exam_time >= break_start1 and exam_time <= break_end1: # check if teacher is free in both times slots
                    available_slot1 = False
                if exam_time >= break_start2 and exam_time <= break_end2:
                    available_slot2 = False

            if teachers_available_slot2 >= teachers_available_slot1: # if one time slot has more teachers than the other, try filling the other timeslot first to maintain balance.
                if available_slot1:
                    teachers_available_slot1 += 1
                elif available_slot2:
                    teachers_available_slot2 += 1
            else:
                if available_slot2:
                    teachers_available_slot2 += 1
                elif available_slot1:
                    teachers_available_slot1 += 1


        # Ensure at least half the faculty is free in one slot and the rest in the other slot
        if not ( (len(available_teachers) // 2) + 2 > teachers_available_slot1 >= (len(available_teachers) // 2) or (len(available_teachers) // 2) + 2 > teachers_available_slot2 >= (len(available_teachers) // 2) ) and \
            ((teachers_available_slot1) + (teachers_available_slot2) ) != len(available_teachers):
            soft_constraints_dissatisfied.add(soft_constraints[3])
        
        constraints = [ hard_constraints_dissatisfied, soft_constraints_dissatisfied ] # combine both constraints into a single list

        return constraints
 
    def __str__(self):
        return '\n'.join(str(exam) for exam in self.exams) # used to display the Object

# Step 4: Define Genetic Operators
def crossover(parent1, parent2):
    # One-point crossover
    cross_point = random.randint(1, len(parent1.exams) - 1)
    child1 = parent1.exams[:cross_point] + parent2.exams[cross_point:]
    child2 = parent2.exams[:cross_point] + parent1.exams[cross_point:]

    return Schedule(child1), Schedule(child2)

def crossover2(parent1, parent2):
    # multi-point crossover

    num_exams = len(parent1.exams)

    # Select two random crossover points
    point1 = random.randint(1, num_exams - 2)
    point2 = random.randint(point1 + 1, num_exams - 1)

    # Create children by swapping segments
    child1_exams = parent1.exams[:point1] + parent2.exams[point1:point2] + parent1.exams[point2:]
    child2_exams = parent2.exams[:point1] + parent1.exams[point1:point2] + parent2.exams[point2:]

    # Ensure no duplicates and all required exams are included
    # child1_exams = list(set(child1_exams))
    # child2_exams = list(set(child2_exams))

    return Schedule(child1_exams), Schedule(child2_exams)

def crossover3(parent1, parent2):
    num_exams = len(parent1.exams)

    # Select a random subset of exams from parent1
    subset_start = random.randint(0, num_exams // 2 - 1)
    subset_end = random.randint(num_exams // 2, num_exams - 1)

    # Create a subset of exams to retain
    child1_subset = parent1.exams[subset_start:subset_end]
    child2_subset = parent2.exams[subset_start:subset_end]

    # Fill in the remaining exams from the other parent, maintaining the order
    child1_remaining = [exam for exam in parent2.exams if exam not in child1_subset]
    child2_remaining = [exam for exam in parent1.exams if exam not in child2_subset]

    child1_exams = child1_remaining[:subset_start] + child1_subset + child1_remaining[subset_start:]
    child2_exams = child2_remaining[:subset_start] + child2_subset + child2_remaining[subset_start:]

    return Schedule(child1_exams), Schedule(child2_exams)

def crossover4(parent1, parent2):

    offspring1 = copy.deepcopy(parent1.exams)
    offspring2 = copy.deepcopy(parent2.exams)

    # Swap genes between the two parents at the chosen indices
    # for i in range(0, len(parent1.exams) - 1):
    for i in range(0, len(parent1.exams)):
        if random.random() <= 0.5:
            offspring1[i], offspring2[i] = offspring2[i], offspring1[i]

    return Schedule(offspring1), Schedule(offspring2)

def crossover5(parent1, parent2):
    
    offspring1 = copy.deepcopy(parent1.exams)
    offspring2 = copy.deepcopy(parent2.exams)

    for i in range(0, len(parent1.exams)):

        if random.random() <= 0.5:

            # Choose two random indices for gene swapping
            index1 = random.randint(0, len(parent1.exams) - 1)
            index2 = random.randint(0, len(parent1.exams) - 1)

            offspring1[index1], offspring2[index2] = offspring2[index1], offspring1[index2]
            # offspring1[index1], offspring2[index2] = offspring2[index2], offspring1[index1]
            # offspring1[index1], offspring2[index1] = offspring2[index1], offspring1[index1]

    return Schedule(offspring1), Schedule(offspring2)
    

def mutation(schedule):
    # Swap two exams as a simple mutation strategy
    index1 = random.randint(0, len(schedule.exams) - 1)
    index2 = random.randint(0, len(schedule.exams) - 1)
    
    schedule.exams[index1], schedule.exams[index2] = schedule.exams[index2], schedule.exams[index1]
    return schedule

def mutation2(schedule):
    index1 = random.randint(0, len(schedule.exams) - 1)
    index2 = random.randint(0, len(schedule.exams) - 1)

    # Remove exam from original position and insert into new position
    exam = schedule.exams.pop(index1)
    schedule.exams.insert(index2, exam)

    return schedule

def mutation3(schedule):

    mutated_chromosome = copy.deepcopy(schedule.exams)
    # Select a subset of genes
    subset_start = random.randint(0, len(mutated_chromosome) - 1)
    subset_end = random.randint(subset_start, len(mutated_chromosome))
    subset = mutated_chromosome[subset_start:subset_end]
    # Invert the subset
    inverted_subset = subset[::-1]
    # Update the chromosome with the inverted subset
    mutated_chromosome[subset_start:subset_end] = inverted_subset

    return Schedule(mutated_chromosome)

def mutation4(schedule):

    mutated_chromosome = copy.deepcopy(schedule.exams)
    # Select a subset of genes
    subset_size = random.randint(1, len(mutated_chromosome))
    subset_indices = random.sample(range(len(mutated_chromosome)), subset_size)
    subset_values = [mutated_chromosome[i] for i in subset_indices]
    # Shuffle or scramble the values of the subset randomly
    random.shuffle(subset_values)
    # Update the chromosome with the scrambled subset
    for i, index in enumerate(subset_indices):
        mutated_chromosome[index] = subset_values[i]

    return Schedule(mutated_chromosome)


def roulette_wheel_selection(population):
    fitnesses = np.array([sched.fitness() for sched in population])
    fitnesses = fitnesses - np.min(fitnesses) + 1  # Normalization to avoid division by zero later 
    
    total_fitness = np.sum(fitnesses)
    selection_probabilities = fitnesses / total_fitness # Calculate selection probabilities
    
    parents = random.choices(population, selection_probabilities, k=2) # Choose parents randomly based on selection probability
    return parents[0], parents[1]

def roulette_wheel_selection2(population):
    total_fitness = sum(x.fitness() for x in population)
    parents = []
    loop_counter = 0

    while loop_counter < 2:
        pick = random.uniform(0,total_fitness) # pick random threshold
        current =0
        for x in population: # loop through population

            fitness = x.fitness()
            if current + fitness >= pick and x not in parents: # if threshold reached, exit loop with parent
                parents.append(x)
                loop_counter += 1
                break
                
            current += fitness

    return parents[0], parents[1]


# Create initial population and simulate evolution
def create_initial_population(num_individuals, available_times, available_classrooms, available_teachers):
    population = []
    
    for _ in range(num_individuals):
        exams = []
        
        for _, course in courses_df.iterrows():
            flag = False
            course_code = course["Course Code"] # Get the course code

            for exam in exams: # If course code already exists, continue
                if course_code == exam.course_code:
                    flag = True

            if flag:
                continue

            # Set details for exam
            start_time = random.choice(available_times) 
            classroom = random.choice(available_classrooms)
            teacher = random.choice(available_teachers)
            
            # Store exams in a list
            exams.append(Exam(course_code, start_time, classroom, teacher))
        
        # Store exams list in a Schedule object
        population.append(Schedule(exams))
    
    return population

def run_genetic_algorithm(num_generations, population_size, mutation_rate, crossover_rate):
    
    # Define available time slots
    time_slots = [pd.to_datetime(f"2024-04-{i} {j}:00:00") for i in range(1, 6) for j in range(9, 17)] # Define available days
    available_classrooms = [f"C{str(x).zfill(3)}" for x in range(301, 311)] # Define available time slots
    available_teachers = list(teachers_df["Names"]) # Define available teachers
    
    # Create initial population
    population = create_initial_population(population_size, time_slots, available_classrooms, available_teachers)


    
    # Generations loop
    for generation in range(num_generations):

        for chromosome in population:
            chromosome.schfitness = chromosome.fitness() # Initialize fitness of population
        
        for _ in range( population_size // 2 ): # Loop for half of population

            #  Parent Selection
            parent1, parent2 = roulette_wheel_selection(population)
            
            # Crossover
            if random.random() < crossover_rate:
                child1, child2 = crossover4(parent1, parent2)
            else:
                child1, child2 = parent1, parent2

            # Mutation
            if random.random() < mutation_rate:
                child1 = mutation2(child1)
            if random.random() < mutation_rate:
                child2 = mutation2(child2)

            # Initalize children fitness
            child1.schfitness = child1.fitness()
            child2.schfitness = child2.fitness()
            
            # Append children
            population.append(child1)
            population.append(child2)

            population.sort(key=lambda x: x.schfitness, reverse=True) # Sort Schedules by fitness
            population = population[:population_size]   # Slice population to population_size
        

        # for x in population:
        #     print(x.schfitness, end=", ")
        # print()
        # Print best solution fitness at each generation 

        best_fitness = max(sched.fitness() for sched in population) # Get best solution fitness in current population
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")     # Display best solution fitness
    
    # Return the best solution found
    best_schedule = max(population, key=lambda sched: sched.fitness())
    
    return best_schedule


num_generations = 30
population_size = 10
mutation_rate = 0.2
crossover_rate = 0.95

best_schedule = run_genetic_algorithm(num_generations, population_size, mutation_rate, crossover_rate)

print("Best schedule found:")
print(best_schedule)


Generation 1: Best Fitness = -10.0
Generation 2: Best Fitness = -8.5
Generation 3: Best Fitness = -8.5
Generation 4: Best Fitness = -8.5
Generation 5: Best Fitness = -8.5
Generation 6: Best Fitness = -8.5
Generation 7: Best Fitness = -8.5
Generation 8: Best Fitness = -8.5
Generation 9: Best Fitness = -8.5
Generation 10: Best Fitness = -8.5
Generation 11: Best Fitness = -8.5
Generation 12: Best Fitness = -8.5
Generation 13: Best Fitness = -8.5
Generation 14: Best Fitness = -8.5
Generation 15: Best Fitness = -8.5
Generation 16: Best Fitness = -8.5
Generation 17: Best Fitness = -8.5
Generation 18: Best Fitness = -8.5
Generation 19: Best Fitness = -8.5
Generation 20: Best Fitness = -8.5
Generation 21: Best Fitness = -8.5
Generation 22: Best Fitness = -8.5
Generation 23: Best Fitness = -8.5
Generation 24: Best Fitness = -8.5
Generation 25: Best Fitness = -8.5
Generation 26: Best Fitness = -8.5
Generation 27: Best Fitness = -8.5
Generation 28: Best Fitness = -8.5
Generation 29: Best Fitness 

## Display the best schedule

In [16]:
import pandas as pd

# Initialize lists to store exam details
course_codes = []
start_times = []
classrooms = []
invigilating_teachers = []

# Extract exam details from the best_schedule
for exam in best_schedule.exams:
    course_codes.append(exam.course_code)
    start_times.append(exam.start_time)
    classrooms.append(exam.classroom)
    invigilating_teachers.append(exam.invigilating_teacher)

# Create a DataFrame to represent the schedule
schedule_df = pd.DataFrame({
    "Start Time": start_times,
    "Classroom": classrooms,
    "Invigilating Teacher": invigilating_teachers
}, index=course_codes)  # Set "Course Code" as index

# Set the name of the index column
schedule_df = schedule_df.rename_axis("Course Code")

# Display the DataFrame
schedule_df.head(100)


,Start Time,Classroom,Invigilating Teacher
Course Code,,,
CS217,2024-04-02 13:00:00,C310,Waqas Munir
EE227,2024-04-03 15:00:00,C310,Shafaq Riaz
CS211,2024-04-04 12:00:00,C306,Irum Inayat
SE110,2024-04-04 14:00:00,C304,Muhammad Asim
CS118,2024-04-05 10:00:00,C307,Nagina Safdar
CS219,2024-04-03 13:00:00,C307,Asif Naeem
CS220,2024-04-04 14:00:00,C309,Mehboobullah
CS302,2024-04-04 15:00:00,C306,Shams Farooq
CY2012,2024-04-03 16:00:00,C304,Maheen Arshad


## Display Constraints

In [14]:
hard_const, soft_const = best_schedule.constraints_dissatisfied()

print("Hard Constraints Satisfied: ")
print('---------------------------------------')
for index, const in enumerate(hard_constraints):
    if const not in hard_const:
        print(f'{index+1}.', const)

print()

print("Soft Constraints Satisfied: ")
print('---------------------------------------')
for index, const in enumerate(soft_constraints):
    if const not in soft_const:
        print(f'{index+1}.', const)


Hard Constraints Satisfied: 
---------------------------------------
1. An exam will be scheduled for each course.
3. Exams will not be held on weekends.
4. Exams must be between 9 am and 5 pm.
5. Each exam must be invigilated by a teacher.
6. A teacher cannot invigilate two exams at the same time.
7. A teacher cannot invigilate two exams in a row.

Soft Constraints Satisfied: 
---------------------------------------
1. All students and teachers shall have a break on Friday from 1-2.
4. Faculty needs a two-hour break in the week.


In [15]:
hard_const, soft_const = best_schedule.constraints_dissatisfied()

print("Hard Constraints Dissatisfied: ")
print('---------------------------------------')
for index, const in enumerate(hard_const):
        print(f'{index+1}.', const)

print()

print("Soft Constraints Dissatisfied: ")
print('---------------------------------------')
for index, const in enumerate(soft_const):
        print(f'{index+1}.', const)


Hard Constraints Dissatisfied: 
---------------------------------------
1. A student cannot give more than one exam at a time.

Soft Constraints Dissatisfied: 
---------------------------------------
1. If a student is enrolled in both MG and CS courses, it's preferred that MG course exam be held before the CS course exam.
2. A student shall not give more than one exam consecutively.
